In [132]:
import numpy as np
import pandas as pd
import json
from datetime import timedelta
import requests

In [133]:
DATA_URL = "http://ec2-108-137-45-5.ap-southeast-3.compute.amazonaws.com/api/netmon/rfcdata/"

In [134]:
res = requests.get(DATA_URL)
wpms_json = res.json()

In [135]:
with open("data.json", "w") as jsonfile:
    json.dump(wpms_json,jsonfile)

In [147]:
df = pd.read_json("data.json")

In [148]:
df['device'] = df['device'].apply(lambda x: x['name'])

In [149]:
def calculate_seconds(dt):
    if not dt.minute % 10:
        return dt.second
    return (dt.minute % 10) * 60 + dt.second

In [150]:
df['timestamp'] = df['timestamp'].apply(lambda d: d - timedelta(seconds=calculate_seconds(d)))

In [168]:
data = df[['device','mode','timestamp', 'rtt', 'bb', 'bdp', 'rwnd', 'rcvbuf_actual', 'actual_thpt', 'tcp_efficiency']]

In [169]:
data = data[data['timestamp'] >= '2022-08-03']

In [170]:
groups = data.groupby(['device', 'mode'])

In [174]:
data[data['actual_thpt'] <= 60000000 ].max()

device                     RPI Malvar
mode                          reverse
timestamp         2022-08-06 15:20:00
rtt                          0.895832
bb                           49361567
bdp                          38486317
rwnd                          7848960
rcvbuf_actual                15697920
actual_thpt                  50481905
tcp_efficiency                  100.0
dtype: object

In [167]:
with open('wpms_data_all.csv', 'w') as wpmscsv:
    data.to_csv(wpmscsv)

In [181]:
malvar_forward = groups.get_group(('RPI Malvar','forward'))
malvar_reverse = groups.get_group(('RPI Malvar','reverse'))

In [182]:
cuenca_forward = groups.get_group(('RPI Cuenca','forward'))
cuenca_reverse = groups.get_group(('RPI Cuenca','reverse'))

In [183]:
with open('malvar_forward.csv', 'w')as malvarforwardcsv:
    malvar_forward.to_csv(malvarforwardcsv)

In [184]:
with open('malvar_reverse.csv', 'w')as malvarreversecsv:
    malvar_reverse.to_csv(malvarreversecsv)

In [185]:
with open('cuenca_forward.csv', 'w')as cuencaforwardcsv:
    cuenca_forward.to_csv(cuencaforwardcsv)

In [186]:
with open('cuenca_reverse.csv', 'w')as cuencareversecsv:
    cuenca_reverse.to_csv(cuencareversecsv)